<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza y Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano y Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Matías Lopez
- Nombre de alumno 2: Jose I. Saffie


### **Link de repositorio de GitHub:** `https://github.com/JoseSaffie/MDS7202-1-Primavera-2023`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.csv`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_players = pd.read_csv('stats_players.csv')

## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
### Código aquí ###
df_players['National_Position'] = np.where(df_players['National_Position'].isnull(), 0, 1)

In [5]:
df_players['National_Position'].value_counts()

National_Position
0    16513
1     1075
Name: count, dtype: int64

Notamos que hay un gran desbalance en los datos, existen 1075 personas que son seleccionados y 16513 no seleccionados, lo que nos dice que esta desbalanceada la clase

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [x] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [x] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [x] Crear 3 ``pipelines`` con diferentes clasificadores.
- [x] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [x] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [6]:
### Creamos los columnTransformer con los preprocesamientos correspondientes
# Antes separamos las columnas categoricas y las numericas, sin tomar el nombre

# Filtramos por int y float para obtner los numericos
numeric_columns = df_players.select_dtypes(include=['int64', 'float64']).columns

# Filtramos por ebjeto, dejamos nombre fuera
categorical_columns = df_players.select_dtypes(include=['object']).columns
categorical_columns = categorical_columns.difference(['Name'])


# Para los valores numericos vamos a utlizar un standarScaler ya que funciona para las variables numericas
# Para los valores categoricos nos interesa utilizar un onehotencoder para capturar cada categoria
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder( handle_unknown='ignore'), categorical_columns)
    ])


# Creamos 3 Pipelines con clasificadores diferentes
# Revisando la guia vemos que tenemos menos de 100k de filas, es una clasificación, por lo que probaremos con SVC, que si no funciona podemos probar con KNN y si no un método
# de ensamblaje como RF. Vemos también hay más de 50 filas, predice una categoria y tiene la data con labels.
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

pipeline_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])

pipeline_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

**Respuesta:**

In [7]:
### Código aquí ###

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [x] Separar el conjunto de datos en entrenamiento y de prueba.
- [x] Entrenar los `pipelines`.
- [x] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [8]:
### Código aquí ###

X = df_players.drop('National_Position', axis=1)  # axis=1 indica que estamos eliminando una columna
y = df_players['National_Position']

# Supongamos que X es tu conjunto de datos y "Seleccion" es tu variable dependiente (target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_players['National_Position'])

# Ahora, entrenamos los pipelines
pipeline_rf.fit(X_train, y_train)
pipeline_svc.fit(X_train, y_train)
pipeline_knn.fit(X_train, y_train)

# Evaluamos los modelos
def evaluate_model(pipeline, X_test, y_test):
    y_pred = pipeline.predict(X_test)
    report = classification_report(y_test, y_pred)
    return report

# Evaluamos cada modelo
print("Random Forest Classifier:")
print(evaluate_model(pipeline_rf, X_test, y_test))

print("\nSupport Vector Classifier:")
print(evaluate_model(pipeline_svc, X_test, y_test))

print("\nK-Nearest Neighbors:")
print(evaluate_model(pipeline_knn, X_test, y_test))

Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.72      0.11      0.19       215

    accuracy                           0.94      3518
   macro avg       0.83      0.55      0.58      3518
weighted avg       0.93      0.94      0.92      3518


Support Vector Classifier:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       1.00      0.01      0.02       215

    accuracy                           0.94      3518
   macro avg       0.97      0.50      0.49      3518
weighted avg       0.94      0.94      0.91      3518


K-Nearest Neighbors:
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      3303
           1       0.53      0.09      0.15       215

    accuracy                           0.94      3518
   macro avg       0.74      0.54      0.56      351

**Respuesta**:


```
Accuracy: Proporción de instancias correctamente clasificadas sobre el total de instancias. Mide la habilidad general del modelo para predecir correctamente.
Precision: Proporción de verdaderos positivos sobre la suma de verdaderos positivos y falsos positivos. Mide la precisión del modelo cuando predice que el jugador es seleccionado.
Recall: Proporción de verdaderos positivos sobre la suma de verdaderos positivos y falsos negativos. Mide la capacidad del modelo para capturar todos los seleccionados.

En cuanto a la revision de los modelos en especifico tenemos que dentro de los 3 distintos clasificadores que se usaron SVM, dio pesimos resultados con donde para la clase seleccionado (1), tiene 0.01 de recall, osea es pesimo identificando a esta clase, por otro lado RF tiene un recall 0.11, un poco mejor y el resto de metricas para estos dos modelos es super alta, para knn vemos un 0.09.

Finalmente nos quedamos con RF ya que tiene un mejor Macro AVG que el resto de modelos, revisando el F1-score, , probablemente la forma de aumentar el el rendimiento de los modelos, seria modificando los hiperparametros y solucionando el problema de las clases desbalanceadas, también el onehotencoder termina creando muchas variables, lo que podria dificultar algunos modelos. Entonces se podrian mejorar los resultados ajustando los hiperparámetros de los modelos, probando diferentes algoritmos, o incluso utilizando técnicas de selección de características.

Y la cantidad de ejemplos por clase influye en el rendimiento del modelo, especialmente si hay un desequilibrio en la distribución de clases. En casos de desequilibrio, es posible que el modelo tenga un rendimiento deficiente para la clase minoritaria, como se puede observar claramente en los datos y resultados obtenidos.

```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [X] Comente los resultados obtenidos.

**Respuesta:**

In [9]:
### Código aquí ###
#
df_players2 = df_players.copy()

# Definir las categorías
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']

# Función para asignar las etiquetas para utilizar pandas de manera eficiente
def asignar_etiqueta(posicion):
    if posicion in ataque:
        return 'Ataque'
    elif posicion in central_ataque:
        return 'Central Ataque'
    elif posicion in central:
        return 'Central'
    elif posicion in central_defensa:
        return 'Central Defensa'
    elif posicion in defensa:
        return 'Defensa'
    elif posicion in arquero:
        return 'Arquero'
    else:
        return 'Otro'

# Aplicar la función a la columna Club_Position y crear una nueva columna llamada 'Categoria' y eliminamos aquellos registros que son sub y res
df_players2['Club_Position'] = df_players2['Club_Position'].apply(asignar_etiqueta)
df_players2 = df_players2[df_players2['Club_Position'] != 'Otro']

################# Con esto tenemos listo
print(df_players2['Club_Position'].value_counts())

# Defensa            1180
#Central             907
#Arquero             632
#Central Ataque      581
#Ataque              430
#Central Defensa     209


### Ahora vamos a entrenar los nuevos pipelines, ya que también es un problema de clasificación igual que el anterior vamos a utilizar los mismo modelos



### Creamos los columnTransformer con los preprocesamientos correspondientes
# Antes separamos las columnas categoricas y las numericas, sin tomar el nombre

# Filtramos por int y float para obtner los numericos
numeric_columns = df_players2.select_dtypes(include=['int64', 'float64']).columns

# Filtramos por ebjeto, dejamos nombre fuera
categorical_columns = df_players2.select_dtypes(include=['object']).columns
categorical_columns = categorical_columns.difference(['Name'])
categorical_columns = categorical_columns.difference(['Club_Position'])


# Para los valores numericos vamos a utlizar un standarScaler ya que funciona para las variables numericas
# Para los valores categoricos nos interesa utilizar un onehotencoder para capturar cada categoria
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder( handle_unknown='ignore'), categorical_columns)
    ])


# Creamos 3 Pipelines con clasificadores diferentes
# Revisando la guia vemos que tenemos menos de 100k de filas, es una clasificación, por lo que probaremos con SVC, que si no funciona podemos probar con KNN y si no un método
# de ensamblaje como RF
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

pipeline_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

pipeline_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])

pipeline_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])


X = df_players2.drop('Club_Position', axis=1)  # axis=1 indica que estamos eliminando una columna
y = df_players2['Club_Position']

# Supongamos que X es tu conjunto de datos y "Seleccion" es tu variable dependiente (target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_players2['Club_Position'])

# Ahora, entrenamos los pipelines
pipeline_rf.fit(X_train, y_train)
pipeline_svc.fit(X_train, y_train)
pipeline_knn.fit(X_train, y_train)

# Evaluamos los modelos
def evaluate_model(pipeline, X_test, y_test):
    y_pred = pipeline.predict(X_test)
    report = classification_report(y_test, y_pred)
    return report

# Evaluamos cada modelo
print("Random Forest Classifier:")
print(evaluate_model(pipeline_rf, X_test, y_test))

print("\nSupport Vector Classifier:")
print(evaluate_model(pipeline_svc, X_test, y_test))

print("\nK-Nearest Neighbors:")
print(evaluate_model(pipeline_knn, X_test, y_test))

Club_Position
Defensa            1180
Central             907
Arquero             632
Central Ataque      581
Ataque              430
Central Defensa     209
Name: count, dtype: int64
Random Forest Classifier:
                 precision    recall  f1-score   support

        Arquero       1.00      1.00      1.00       126
         Ataque       0.75      0.85      0.80        86
        Central       0.58      0.66      0.62       182
 Central Ataque       0.52      0.28      0.37       116
Central Defensa       0.33      0.10      0.15        42
        Defensa       0.81      0.96      0.88       236

       accuracy                           0.74       788
      macro avg       0.66      0.64      0.63       788
   weighted avg       0.71      0.74      0.71       788


Support Vector Classifier:
                 precision    recall  f1-score   support

        Arquero       1.00      1.00      1.00       126
         Ataque       0.79      0.86      0.82        86
        Central  

**Respuesta**:

```
Viendo los resultados lo primero que observamos que de los 3 modelos propuestos encontramos que el peor de los 3 es el de RF, en orden de clases que mejor logran reconocer los modelos vemos que arquero es el más facil de reconocer, después defensa, después ataque y luego central. Los modelos les cuesta mucho reconocer central ataque y central defensa, todo esto se reviso viendo el f1-score. Finalmente si comparamos el f1-score y acc de los modelos de knn y svm, vemos que el svm es un poco mejor ya que tiene un 1 puntos más en acc y en weighted avg. Los modelos logran resultados satisfactorios dentro de todo, ya que son varias las clase y esto dificulta el entrenamiento y despues testeo del modelo. 

Nuevamente, si las clases fueran mas balanceadas se lograrian mejores resultados y quizas generalizar central ataque y central defensa que son aquellos que estan generando problemas y asignar estas clases a otra clases para generalizar resultados y que mejore el modelo en general.

pd: clasificar arquero tiene un desempeño del 100% para los 3 modelos.

pd2: recalcar que se utilizaron los mismos pipelines que en la parte anterior dado que también es un problema de clasificación
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

|Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [X] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [10]:
df_players = pd.read_csv('stats_players.csv')
df_salarios = pd.read_csv('salarios.csv')

df_full = pd.merge(df_players, df_salarios, left_on='Name',right_on='Player', how='inner')
df_full

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Unnamed: 0,Player,Weekly Salary
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,92,93,90,81,76,85,88,1006,Cristiano Ronaldo,1248536.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,85,95,88,89,90,74,85,1528,Lionel Messi,1538905.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,78,89,77,79,84,81,83,2004,Neymar,797726.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,87,94,86,86,84,85,88,1531,Luis Suárez,508923.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,25,13,16,14,11,47,11,498,Manuel Neuer,326233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Phillip Menzel,Germany,NaN,Res,191,83,Right,18,Medium / Medium,3,...,25,5,10,12,8,15,5,973,Phillip Menzel,2034.0
1862,Manuel Akanji,Switzerland,NaN,Sub,187,85,Right,21,Medium / Medium,2,...,38,25,18,26,27,40,31,600,Manuel Akanji,54176.0
1863,Moritz Nicolas,Germany,NaN,Res,195,87,Right,19,Medium / Medium,2,...,22,9,10,17,14,17,11,965,Moritz Nicolas,2262.0
1864,Giacomo Satalino,Italy,NaN,Sub,188,70,Right,17,Medium / Medium,1,...,19,6,5,13,11,18,9,1508,Giacomo Satalino,2827.0


In [11]:
### Código aquí ###

# Antes vamos a determinar el modelo que nos sirve, en este caso vemos que es un modelo de regresion, a diferencia del anterior que era de clasificacion. Entonces, tenemos mas de 50 datos,
# no predice una categoria, predice una cantidad y hay menos de 100k de filas. Entonces podemos probar con ElasticNet, Lasso, ridge regression o SVR(kernel = linear).

# Dividir los datos en características (X) y variable objetivo (y)
X = df_full.drop('Weekly Salary', axis=1)
y = df_full['Weekly Salary']

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Filtramos por int y float para obtner los numericos
numeric_columns = df_full.select_dtypes(include=['int64', 'float64']).columns
numeric_columns = numeric_columns.difference(['Weekly Salary'])


# Filtramos por ebjeto, dejamos nombre fuera
categorical_columns = df_full.select_dtypes(include=['object']).columns
categorical_columns = categorical_columns.difference(['Name'])

# Combinar transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder( handle_unknown='ignore'), categorical_columns)
    ])


from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression, SGDRegressor

pipeline_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LinearRegression())
])

pipeline_en = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', ElasticNet())
])

pipeline_LASSO = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', Lasso())
])

pipeline_RIDGE = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', Ridge())
])

pipeline_SGD = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDRegressor())
])

# Entrenar y evaluar cada modelo
from sklearn.metrics import r2_score
pipeline_lr.fit(X_train,y_train)
y_pred = pipeline_lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'Linear Regression - R^2 Score: {r2}')

pipeline_en.fit(X_train,y_train)
y_pred = pipeline_en.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'ElasticNet - R^2 Score: {r2}')

pipeline_LASSO.fit(X_train,y_train)
y_pred = pipeline_LASSO.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'Lasso - R^2 Score: {r2}')

pipeline_RIDGE.fit(X_train,y_train)
y_pred = pipeline_RIDGE.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'Ridge - R^2 Score: {r2}')

pipeline_SGD.fit(X_train,y_train)
y_pred = pipeline_SGD.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'SGD Regressor - R^2 Score: {r2}')

Linear Regression - R^2 Score: 0.35231733836907797
ElasticNet - R^2 Score: 0.21611482934992976
Lasso - R^2 Score: 0.49211136306962333
Ridge - R^2 Score: 0.30070916907159473
SGD Regressor - R^2 Score: 0.11481689539164397


c:\Users\josei\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 38384279722.47841, tolerance: 1137978847.1056485
  model = cd_fast.sparse_enet_coordinate_descent(


**Respuesta**:

```
La métrica de  R2, corresponde al coeficiente de determinación de una regresión, permite ver la proporción en la variable dependiente que se predice a partir de la variable independiente, que puede tomar valores del 1 al 0, donde 0 indica que las variables independientes no explican nada la variabilidad de la variable dependiente y donde 1, nos indica que las variables independientes explican las variabilidad perfectamente a la variable dependiente. Es decir, entre más cerca a 1, mejor es el modelo.

En este caso vemos que es un modelo de regresión, a diferencia del anterior que era de clasificación. Entonces, tenemos mas de 50 datos, no predice una categoría, predice una cantidad y hay menos de 100k de filas. Entonces podemos probar con ElasticNet, Lasso, ridge regression o SVR(kernel = linear).

Notamos que logramos obtener dos modelos con R2 superior a 0,35, el de Lasso (0.492) y la regresión lineal (0.352), dejando a lasso como el mejor modelo para estos datos.

Para mejorar los resultados, podemos ajustar los hiperparámetros de lasso o probar con el ensamblaje de varios modelos. También se podría incluir un feature engineering para agregar variables que podrían explicar mejor aun la variabilidad de la variable dependiente. Por ultimo métodos de cross-validation podrían influir en un mejor entrenamiento y por tanto un mejor modelo en general. En general la suma de mas filas seria de ayuda para poder probar modelos mas complejos y terminar con un mejor modelo al final, también se podrían incluir más características, que probablemente ayudan a predecir mejor finalmente.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>